In [3]:
using Logging
global_logger(ConsoleLogger());

In [4]:
## Load the code

include("N_point_NCG_PEP.jl");

In [5]:
## Parameters

μ = 0.5

L = 1

R = 1

N = 2

η_input = 1 # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR

ξ_input = 0 # controls restarting scheme: 
# if ξ = 1 => NCG method is restarted at iteration k(=0 in code), 
# if ξ = 0 => we just use a bound of the form ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2

c_0_input = (L+μ)^2/(4*μ*L) # controls ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2

1.125

## Solve the NCG-PEP with line search relaxed to find an upper bound to the contraction factor $\frac{f(x_N)-f(x_\star)}{f(x_0) - f(x_\star)}$

In [6]:
## Generate a feasible solution to NCG-PEP
G_ws, Ft_ws, L_cholesky_ws, α_ws,  χ_ws, α_tilde_ws, β_ws = feasible_sol_generator_NCG_PEP(N, μ, L, η_input);

[ Info: norm(p_feas_array - p_test_feas_array)=0.0
[ Info: norm(x_feas_array - x_test_feas_array)=2.2247786310271853e-16


In [7]:
## Compute a locally optimal solution to NCG-PEP

G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, χ_loc_opt, β_loc_opt, contraction_factor_loc_opt, solve_time_NCG_PEP_loc_opt = N_point_NCG_PEP_solver(
    ## paramters
    # ----------
    N, μ, L,
    G_ws, Ft_ws, L_cholesky_ws, χ_ws, β_ws;
    ## options
    # --------
    η = η_input, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
    ξ = ξ_input, # controls restarting scheme: if ξ = 1 => NCG method is restarted at iteration k(=0 in code), if not we just use a bound of the form ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    c_0 = c_0_input, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    solution_type = :find_locally_optimal, # options are :find_locally_optimal and :find_globally_optimal
    ρ_lower_bound = 0, # Lower bound on the contraction factor
    ρ_upper_bound = Inf, # Upper bound on the contraction factor
    impose_pattern = :on, # options are :on and :off, (i) if :on then we use the interpolation inequalities corresponding to the effective index set of λ, and (ii) if :off, we use all the interpolation inequalities
    show_output = :off, # options are :on and :off
    PSDness_modeling = :exact, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    ϵ_tol_feas_Gurobi = 1e-6, # Feasibility of primal constraints, i.e., whether ⟨a;x⟩ ≤ b holds for the primal solution. More precisely, ⟨a;x⟩ ≤ b  will be considered to hold if ⟨a;x⟩ - b ≤  FeasibilityTol. We have a polishing mechanism to improve the tolerance, so the parameter can be set as low as 1e-4
    max_cut_count = 1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
    );

## Warm-start

G_ws, Ft_ws, L_cholesky_ws, χ_ws, β_ws = G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, χ_loc_opt, β_loc_opt;

## Polish the locally optimal solution

G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, χ_loc_opt, β_loc_opt, contraction_factor_loc_opt, solve_time_NCG_PEP_loc_opt = N_point_NCG_PEP_solver(
    ## paramters
    # ----------
    N, μ, L,
    G_ws, Ft_ws, L_cholesky_ws, χ_ws, β_ws;
    ## options
    # --------
    η = η_input, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
    ξ = ξ_input, # controls restarting scheme: if ξ = 1 => NCG method is restarted at iteration k(=0 in code), if not we just use a bound of the form ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    c_0 = c_0_input, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    solution_type = :find_locally_optimal, # options are :find_locally_optimal and :find_globally_optimal
    ρ_lower_bound = 0, # Lower bound on the contraction factor
    ρ_upper_bound = Inf, # Upper bound on the contraction factor
    impose_pattern = :on, # options are :on and :off, (i) if :on then we use the interpolation inequalities corresponding to the effective index set of λ, and (ii) if :off, we use all the interpolation inequalities
    show_output = :on, # options are :on and :off
    PSDness_modeling = :exact, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    ϵ_tol_feas_Gurobi = 1e-6, # Feasibility of primal constraints, i.e., whether ⟨a;x⟩ ≤ b holds for the primal solution. More precisely, ⟨a;x⟩ ≤ b  will be considered to hold if ⟨a;x⟩ - b ≤  FeasibilityTol. We have a polishing mechanism to improve the tolerance, so the parameter can be set as low as 1e-4
    max_cut_count = 1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
    );

[ Info: [🐰 ] locally optimal solution finder activated, solution method: interior point method
[ Info: [🎉 ] defining the variables
[ Info: [🎣 ] imposing pattern on effective index set of λ
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: coninterpol ℓ = i_j_idx(-1, 0)
[ Info: coninterpol ℓ = i_j_idx(-1, 1)
[ Info: coninterpol ℓ = i_j_idx(-1, 2)
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, 2)
[ Info: [👲 ] warm-start values for all the variables
[ Info: 😃 all the variables are warm-started
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
┌ Info: NCG_PEP_model has termination status = 
└   termination_status(NCG_PEP_model) = LOCALLY_SOLVED::TerminationStatusCode = 4
[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 0.0005830117691528556, and objective value of found solution = 0.056124256352827376



           Academic License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 13.1.0

Knitro presolve eliminated 23 variables and 24 constraints.

convex:                  0
datacheck:               0
hessian_no_f:            1
honorbnds:               1
maxit:                   50000
strat_warm_start:        1
The problem is identified as a QCQP.
Knitro shifted start point further inside presolved bounds (4 variables).
Knitro fixing 23 variables eliminated from the presolve.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Maximize
Objective type:  linear
Number of variables:                                 83 (          60)
    bounded below only:                               1 (           1)
    bounded above only:                               0 (           0)
    bounded below and above:                         61 (          59)
    fixed:                                           21 (           0)
    free:  

[ Info: [🐰 ] locally optimal solution finder activated, solution method: interior point method
[ Info: [🎉 ] defining the variables
[ Info: [🎣 ] imposing pattern on effective index set of λ
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: coninterpol ℓ = i_j_idx(-1, 0)
[ Info: coninterpol ℓ = i_j_idx(-1, 1)
[ Info: coninterpol ℓ = i_j_idx(-1, 2)
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, 2)
[ Info: [👲 ] warm-start values for all the variables
[ Info: 😃 all the variables are warm-started
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
┌ Info: NCG_PEP_model has termination status = 
└   termination_status(NCG_PEP_model) = LOCALLY_SOLVED::TerminationStatusCode = 4
[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 0.05612425626431034, and objective value of found solution = 0.05612426677555743


In [8]:
## Set new warm-start values

G_ws, Ft_ws, L_cholesky_ws, χ_ws, β_ws = G_loc_opt, Ft_loc_opt, L_cholesky_loc_opt, χ_loc_opt, β_loc_opt

ϵ_tol_feas_lazy_eigval = 1e-6

ϵ_tol_feas_Gurobi = 1e-6

## Globally optimal solution

G_glb_opt, Ft_glb_opt, L_cholesky_glb_opt, χ_glb_opt, β_glb_opt, contraction_factor_glb_opt, solve_time_NCG_PEP_glb_opt = N_point_NCG_PEP_solver(
    ## paramters
    # ----------
    N, μ, L,
    G_ws, Ft_ws, L_cholesky_ws, χ_ws, β_ws;
    ## options
    # --------
    η = η_input, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
    ξ = ξ_input, # # controls restarting scheme: if ξ = 1 => NCG method is restarted at iteration k(=0 in code), if not we just use a bound of the form ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    c_0 = c_0_input, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    solution_type = :find_globally_optimal, # options are :find_locally_optimal and :find_globally_optimal
    ρ_lower_bound = 0, # Lower bound on the contraction factor
    ρ_upper_bound = Inf, # Upper bound on the contraction factor
    impose_pattern = :on, # options are :on and :off, (i) if :on then we use the interpolation inequalities corresponding to the effective index set of λ, and (ii) if :off, we use all the interpolation inequalities
    show_output = :on, # options are :on and :off
    PSDness_modeling = :lazy_constraint_callback, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = ϵ_tol_feas_lazy_eigval, # feasiblity tolerance for minimum eigenvalue of G
    ϵ_tol_feas_Gurobi = ϵ_tol_feas_Gurobi, # Feasibility of primal constraints, i.e., whether ⟨a;x⟩ ≤ b holds for the primal solution. More precisely, ⟨a;x⟩ ≤ b  will be considered to hold if ⟨a;x⟩ - b ≤  FeasibilityTol. We have a polishing mechanism to improve the tolerance, so the parameter can be set as low as 1e-4
    max_cut_count = 1e6 # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
    )

[ Info: [🐌 ] globally optimal solution finder activated, solution method: spatial branch and bound
[ Info: [🎉 ] defining the variables


Set parameter Username
Academic license - for non-commercial use only - expires 2023-11-06
Set parameter NonConvex to value 2
Set parameter MIPFocus to value 3
Set parameter MIPGap to value 0.01


[ Info: [🎣 ] imposing pattern on effective index set of λ
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: coninterpol ℓ = i_j_idx(-1, 0)
[ Info: coninterpol ℓ = i_j_idx(-1, 1)
[ Info: coninterpol ℓ = i_j_idx(-1, 2)
[ Info: coninterpol ℓ = i_j_idx(0, 1)
[ Info: coninterpol ℓ = i_j_idx(1, 2)


Set parameter MIPFocus to value 1


[ Info: [👲 ] warm-start values for all the variables
[ Info: 😃 all the variables are warm-started
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
┌ Info: NCG_PEP_model has termination status = 
└   termination_status(NCG_PEP_model) = OPTIMAL::TerminationStatusCode = 1


Set parameter NonConvex to value 2
Set parameter MIPFocus to value 1
Set parameter MIPGap to value 0.01
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 376 rows, 83 columns and 7398 nonzeros
Model fingerprint: 0xe0730612
Model has 3 quadratic constraints
Coefficient statistics:
  Matrix range     [3e-07, 4e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e-01, 6e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 70 rows and 25 columns

Continuous model is non-convex -- solving as a MIP


Loaded user MIP start with objective 0.0561243

Presolve removed 76 rows and 27 columns
Presolve time: 0.00s
Presolved: 308 rows, 56 columns, 6224 nonzeros
Presolved model has 2 bilinear constraint(s)
Variable types: 56 continuous, 0 in

[ Info: [😻 ] optimal solution found done, store the solution
┌ Warning: [🍡 ] smallest normalized eigenvalue of the PSD matrix is -2.2275813869433187e-7
└ @ Main ~/My Drive/GitHub/NCG-PEP-code/Code_for_NCG_PEP/code_to_compute_pivoted_cholesky.jl:75
[ Info: [💹 ] warm-start objective value = 0.056124266745714774, and objective value of found solution = 0.056125604591201826


([1.3940904687829125 1.2391915278070333 … -0.15000174137840855 -0.3339923313874612; 1.2391915278070333 1.2391915278070333 … 0.19172594091132958 -0.22915407047454245; … ; -0.15000174137840855 0.19172594091132958 … 0.7843742385141618 0.20075956108975024; -0.3339923313874612 -0.22915407047454245 … 0.20075956108975024 0.14327339638992545], [1.0, 0.26645566799656434, 0.056125604591201826], [1.1807161331034752 0.0 … 0.0 0.0; 1.0495253668798612 0.37106344506019073 … 0.0 0.0; … ; -0.12704305324080736 0.8760246842021047 … 0.0 0.0; -0.28287271932610697 0.18252450183879093 … 0.0 0.0], 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 0:0
    Dimension 2, Base.OneTo(1)
And data, a 1×1 Matrix{Float64}:
 0.11033881547533166, 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:0
And data, a 1-element Vector{Float64}:
 0.11033881547533166, 0.056125604591201826, 2.317898988723755)

## Compute an lower bound to the contraction factor $\frac{f(x_N)-f(x_\star)}{f(x_0) - f(x_\star)}$ and also a bad function where the NCG in consideration does not work well

In [9]:
## Finding the worst-case function
# ================================

## Warm start for exact line search
G_ws_els, Ft_ws_els, L_cholesky_ws_els, γ_ws_els,  χ_ws_els, α_ws_els, β_ws_els = feasible_sol_generator_NCG_PEP_exact_line_search(N, μ, L, η_input)

## Compute the locally optimal solution for exact line search

I_N_star = -1:N

idx_set_λ_ws_effective = index_set_constructor_for_dual_vars_full(I_N_star)

## Compute the locally optimal solution for exact line search

I_N_star = -1:N

idx_set_λ_ws_effective = index_set_constructor_for_dual_vars_full(I_N_star)

G_loc_opt_els, Ft_loc_opt_els, L_cholesky_loc_opt_els, γ_loc_opt_els,  χ_loc_opt_els, α_loc_opt_els, β_loc_opt_els = NCG_PEP_exact_line_search_solver(
    # different parameters to be used
    # -------------------------------
    N, μ, L, R, idx_set_λ_ws_effective,
    # warm-start values for the variables
    G_ws_els, Ft_ws_els, L_cholesky_ws_els, γ_ws_els,  χ_ws_els, α_ws_els, β_ws_els
    ;
    # options
    # -------
    η = η_input, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
    ξ = ξ_input, # # controls restarting scheme: if ξ = 1 => NCG method is restarted at iteration k(=0 in code), if not we just use a bound of the form ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    c_0 = c_0_input, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    bound_impose = :off,
    solution_type =  :find_locally_optimal, # other option :find_globally_optimal
    show_output = :off, # other option :on
    local_solver = :knitro, # other option :knitro
    knitro_multistart = :off, # other option :on (only if :knitro solver is used)
    knitro_multi_algorithm = :off, # other option on (only if :knitro solver is used)
    upper_bound_opt_val = Inf, # this is coming from Borys Polyak's paper
    lower_bound_opt_val = 0,
    reduce_index_set_for_λ = :on,
    PSDness_modeling = :exact, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    maxCutCount=1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
    fix_β = :on, # options are :on and :off
    β_fixed = β_loc_opt
    );

## Set new warm-start values and polish the locally optimal solution for exact line search

G_ws_els, Ft_ws_els, L_cholesky_ws_els, γ_ws_els,  χ_ws_els, α_ws_els, β_ws_els = 
G_loc_opt_els, Ft_loc_opt_els, L_cholesky_loc_opt_els, γ_loc_opt_els, χ_loc_opt_els, α_loc_opt_els, β_loc_opt_els;


G_loc_opt_els, Ft_loc_opt_els, L_cholesky_loc_opt_els, γ_loc_opt_els,  χ_loc_opt_els, α_loc_opt_els, β_loc_opt_els = NCG_PEP_exact_line_search_solver(
    # different parameters to be used
    # -------------------------------
    N, μ, L, R, idx_set_λ_ws_effective,
    # warm-start values for the variables
    G_ws_els, Ft_ws_els, L_cholesky_ws_els, γ_ws_els,  χ_ws_els, α_ws_els, β_ws_els
    ;
    # options
    # -------
    η = η_input, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
    ξ = ξ_input, # # controls restarting scheme: if ξ = 1 => NCG method is restarted at iteration k(=0 in code), if not we just use a bound of the form ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    c_0 = c_0_input, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
    bound_impose = :off,
    solution_type =  :find_locally_optimal, # other option :find_globally_optimal
    show_output = :on, # other option :on
    local_solver = :knitro, # other option :knitro
    knitro_multistart = :off, # other option :on (only if :knitro solver is used)
    knitro_multi_algorithm = :off, # other option on (only if :knitro solver is used)
    upper_bound_opt_val = Inf, # this is coming from Borys Polyak's paper
    lower_bound_opt_val = 0,
    reduce_index_set_for_λ = :on,
    PSDness_modeling = :exact, # options are :exact and :through_ϵ and :lazy_constraint_callback
    ϵ_tol_feas = 1e-6, # feasiblity tolerance for minimum eigenvalue of G
    maxCutCount=1e6, # number of lazy cuts when we use lazy constraints for modeling G = L_cholesky*L_cholesky
    fix_β = :on, # options are :on and :off
    β_fixed = β_loc_opt
    );

## Warm-start again

G_ws_els, Ft_ws_els, L_cholesky_ws_els, γ_ws_els,  χ_ws_els, α_ws_els, β_ws_els = 
G_loc_opt_els, Ft_loc_opt_els, L_cholesky_loc_opt_els, γ_loc_opt_els, χ_loc_opt_els, α_loc_opt_els, β_loc_opt_els;

## Time to solve the SDP solution to the exact line search and generate the worst-case function

p_star_SDP, G_star_SDP, Ft_star_SDP, L_cholesky_star_SDP = SDP_with_fixed_stepsize_NCG_PEP_exact_line_search(
                # Inputs
                # ======
                N, μ, L, R, γ_loc_opt_els,  χ_loc_opt_els, α_loc_opt_els, β_loc_opt_els,
                # Warm-start points
                G_ws_els, Ft_ws_els; # [🐯 ] cost coefficients
                # Options
                # =======
                η = η_input, # controls PRP or FR, if η = 1 => PRP, and if η = 0 => FR
                ξ = ξ_input, # # controls restarting scheme: if ξ = 1 => NCG method is restarted at iteration k(=0 in code), if not we just use a bound of the form ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
                c_0 = c_0_input, # decides ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
                show_output = :on, # options are :on and :off
                ϵ_tolerance = 1e-6,
                reduce_index_set_for_λ = :off)

# Generate the worst-case function

wf = generate_worst_case_function_NCG_PEP_exact_line_search(
N, G_star_SDP, Ft_star_SDP,  L_cholesky_star_SDP, γ_loc_opt_els,  χ_loc_opt_els, α_loc_opt_els, β_loc_opt_els;
ξ = ξ_input
# controls restarting scheme: if ξ = 1 => NCG method is restarted at iteration k(=0 in code), if ξ = 0 then we just use a bound of the form
# ||g_0||^2 <= ||p_0||^2 <= c_0 ||g_0||^2
)

[ Info: norm(p_feas_array - p_test_feas_array)=0.0
[ Info: norm(x_feas_array - x_test_feas_array)=1.3096875034602268e-16
[ Info: [🐙 ] locally optimal solution finder activated, solution method: interior point method
[ Info: [🚀 ] activating KNITRO
[ Info: [🎉 ] defining the variables
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: [🎇 ] adding objective
[ Info: [👲 ] warm-start values for all the variables
[ Info: [💁 ] Fixing β and finding the corresponding solution
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
┌ Info: NCG_PEP_exact_model has termination status = 
└   termination_status(NCG_PEP_exact_model) = LOCALLY_SOLVED::TerminationStatusCode = 4
[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 0.001066094653878792, and objective value of found solution = 0.05610378747833845



           Academic License
       (NOT FOR COMMERCIAL USE)
         Artelys Knitro 13.1.0

Knitro presolve eliminated 78 variables and 85 constraints.

convex:                  0
datacheck:               0
hessian_no_f:            1
honorbnds:               1
strat_warm_start:        1
The problem is identified as a QCQP.
Knitro shifted start point further inside presolved bounds (1 variable).
Knitro fixing 78 variables eliminated from the presolve.

Problem Characteristics                                 (   Presolved)
-----------------------
Objective goal:  Maximize
Objective type:  linear
Number of variables:                                127 (          49)
    bounded below only:                               4 (          11)
    bounded above only:                               0 (           0)
    bounded below and above:                          0 (           2)
    fixed:                                           11 (           0)
    free:                                  

[ Info: [🐙 ] locally optimal solution finder activated, solution method: interior point method
[ Info: [🚀 ] activating KNITRO
[ Info: [🎉 ] defining the variables
[ Info: [🎍 ] adding the data generator function to create 𝐱, 𝐠, 𝐟
[ Info: [🎢 ] adding the constraints
[ Info: [🎇 ] adding objective
[ Info: [👲 ] warm-start values for all the variables
[ Info: [💁 ] Fixing β and finding the corresponding solution
[ Info: [🙌 	🙏 ] model building done, starting the optimization process
┌ Info: NCG_PEP_exact_model has termination status = 
└   termination_status(NCG_PEP_exact_model) = LOCALLY_SOLVED::TerminationStatusCode = 4
[ Info: [😻 ] optimal solution found done, store the solution
[ Info: [💹 ] warm-start objective value = 0.05610378747833845, and objective value of found solution = 0.05610378747833845
[ Info: [🎉 ] defining the variables
[ Info: [🎢 ] adding the constraints
[ Info: [👲 ] warm-start values for all the variables
[ Info: Time to optimize


Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 22              
  Cones                  : 0               
  Scalar variables       : 3               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimizat

[ Info: [😈 ] Optimal solution found for solve_primal_with_known_stepsizes
[ Info: [🌞 ] PRIMALSDP-POLISHED ADAPEP contraction factor V_{k+1}/V_{k-1} = 0.05610378224149573


worst_case_function([0.0 1.6726016549945675 0.35408577303074074 -0.14082540467200277; 0.0 0.0 -0.810307272303649 -0.3229589421760931; … ; 0.0 0.0 0.000223350057601495 0.00024575515071429007; 0.0 0.0 0.0 0.0], [0.0 1.0873397132945846 0.30336080510228297 -0.1585721601231999; 0.0 0.23720597130669774 -0.4756736382672492 -0.2055181552403833; … ; 0.0 0.0 0.0 0.00016705333337364953; 0.0 0.0 0.0 0.0], [0.0, 0.9999999271122465, 0.26734951912074323, 0.05610378224149573])